BLG 453E - Computer Vision<br>Homework 4<br>Muhammed Tolga Cangöz<br>150130024

In [9]:
!pip install ninja
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html



 

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
from argparse import Namespace

import numpy as np
import torch
import sys
import dlib
sys.path.append(".")
sys.path.append("..")

from options.test_options import TestOptions
from models.psp import pSp
import cv2
from scripts.align_all_parallel import align_face
import torchvision.transforms as transforms

#test_opts = TestOptions().parse()

test_opts = Namespace(checkpoint_path=None, couple_outputs=False, data_path='gt_images', exp_dir=None, latent_mask=None, mix_alpha=None, n_images=None, n_outputs_to_generate=5, resize_factors=None, resize_outputs=False, test_batch_size=2, test_workers=2)


test_opts.checkpoint_path = 'pretrained_models/psp_ffhq_encode.pt'


# update test options with options used during training
ckpt = torch.load(test_opts.checkpoint_path, map_location='cpu')
opts = ckpt['opts']
opts.update(vars(test_opts))
opts['output_size'] = 1024

opts = Namespace(**opts)



net = pSp(opts)
net.eval()
net.cuda()
#Prepare the PSPNet + StyleGAN network

transform = transforms.Compose([
	transforms.Resize(size=(256, 256)),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])]
)
#Preprocessing operations for each image

predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
#Predict facial landmarks

aligned_image1 = align_face(filepath="nuri.jpg", predictor=predictor)
aligned_image1 = aligned_image1.convert("RGB")
# Align face using the landmarks

from_im1 = transform(aligned_image1).unsqueeze(0)
# Do preprocessing on the aligned face

aligned_image2 = align_face(filepath="albert.jpeg", predictor=predictor)
aligned_image2 = aligned_image2.convert("RGB")
# Align face using the landmarks

from_im2 = transform(aligned_image2).unsqueeze(0)
# Do preprocessing on the aligned face

with torch.no_grad():
	input1 = from_im1.float().cuda()
	_, latent_vector1 = net(input1, randomize_noise=False, resize=False, return_latents=True)
	input2 = from_im2.float().cuda()
	_, latent_vector2 = net(input2, randomize_noise=False, resize=False, return_latents=True)
	# Obtain features from input image

	for i in range(101):
		z3 = ((100-i)/100)*latent_vector1 + (i/100)*latent_vector2
		img3, _ = net.decoder([z3.float()],input_is_latent=True, randomize_noise=False, return_latents=False)
		result = img3.squeeze().permute((1,2,0)).cpu().numpy()

		result[result>1] = 1
		result[result<-1] = -1
		result = (255*(result+1)//2).astype(np.uint8)

		cv2.imwrite("results/result_{:03d}.png".format(i), result[:,:,[2,1,0]])


Loading pSp from checkpoint: pretrained_models/psp_ffhq_encode.pt


In [4]:
import moviepy.editor as mpy

clip = mpy.ImageSequenceClip(os.path.join("results/"), fps=10)
clip.write_videofile('part3_video.mp4')

Moviepy - Building video part3_video.mp4.
Moviepy - Writing video part3_video.mp4



Moviepy - Done !
Moviepy - video ready part3_video.mp4
